# Задание 3: автореферирование текста

## 1) Импорт библиотек

In [1]:
import nltk
import json

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer

from sumy.evaluation.rouge import rouge_2

## 2) Загрузка данных и функция обработки

In [2]:
def summarize(text, summarizer):
    parser = PlaintextParser.from_string(text, Tokenizer('russian'))
    sentences_orig = parser.document.sentences
    
    summary = summarizer(parser.document, 2)
    result = '\n\n'.join([str(sent) for sent in summary])[:300]
    
    parser = PlaintextParser.from_string(result, Tokenizer('russian'))
    sentences_result = parser.document.sentences
    
    score = rouge_2(sentences_result, sentences_orig)
    
    return (result, score)

with open('example_texts.json', 'r', encoding='utf-8') as file:
    texts = json.load(file)

## 3) LexRank

Данный алгоритм представляет собой модификацию популярного TextRank. В его основе так же лежит <a href="https://snap.stanford.edu/class/cs224w-readings/Brin98Anatomy.pdf">PageRank</a>, который в контексте автореферирования применянется для ранжирования текстов.

В отличие от TextRank, LexRank использует другую метрику схожести:

$\large sim_{ij} = \frac{\sum_{w\in\{S_i\cup S_j\}} {tf}_w^i * {tf}_w^j * ({idf}_w)^2}{\sqrt{\sum_{w\in S_i} ({tf}_w^i * {idf}_w)^2} * \sqrt{\sum_{w\in S_j} ({tf}_w^j * {idf}_w)^2}}$

$\large {tf}_w^i = \frac{|\{w_k \mid (w_k \in S_i) \wedge (w_k = w)\}|}{|S_i|}$

$\large {idf}_w = log(\frac{|\mathbb{S}|}{|\{S_i \mid w \in S_i\}|})$

Здесь $\mathbb{S}$ - множество всех предложений, $S_i$ - набор слов $i$-го предложения, $w$ - токены в предложении. Такая метрика позволяет больше учитывать редкие термины вместо того чтобы отдавать предпочтение часто употребляемым словам.

К тому же, веса ребрер между предложениями, имеющими низкую схожесть, обнуляются. Для этого задается пороговое значение $t$ (в реализации <b>sumy</b> это $t=0.1$).

In [3]:
lr_summaries = []

LexRankSummarizer.threshold = 0.05 # 0.1
LexRankSummarizer.epsilon = 0.05 # 0.1

for text in texts:    
    lr_summaries.append(summarize(text, LexRankSummarizer()))

In [4]:
for num, res in enumerate(lr_summaries):
    summ, score = res
    print('\n', end=f'Summary #{num + 1}:\n')
    print(summ)
    print(f'ROUGE-2: {score}')


Summary #1:
Увидеть мысль

За последние несколько лет нейрофизиология сделала большой шаг вперед в понимании работы человеческого мозга .
ROUGE-2: 0.11666666666666667

Summary #2:
Карбофос ( O , O - Диметил - S - ( 1,2 - дикарбэтоксиэтил ) дитиофосфат , малатион ) — фосфорорганическое соединение , инсектицид широкого спектра действия , акарицид .

Механизм действия
ROUGE-2: 0.17045454545454544

Summary #3:
Киев идет на уступки : Яценюк передает власть на местах Советам

Центральная власть готова не просто к диалогу с регионами , а к выполнению законных требований и желаний каждого жителя нашей страны , и мы в рамках изменений Конституции будем в состоянии ответить на все чаяния , на каждую специфику к
ROUGE-2: 0.20657276995305165

Summary #4:
Но дело в том , что , во-первых , срок оплаты за июль ― 10 августа , так что я ещё не платила за июль , и долг не мог образоваться физически !

Немцов , который недавно не смог стать мэром не то что Москвы , а даже маленьких Сочей ( о которых бы 

## 4) Запись результатов в JSON

In [5]:
output_obj = [summ for summ, score in lr_summaries]
ind = '\t'
sep = (',', ' : ')

with open('output_texts.json', 'w', encoding='utf-8') as file:
    json.dump(output_obj, file, ensure_ascii=False, indent=ind, separators=sep)
    file.close()